# Creating a baseline for recomendations

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

## Imports and setup.

In [26]:
boardgames_df = pd.read_csv('data/modern_games.csv')
users_df = pd.read_csv('data/users_encoded.csv')

In [38]:
users_df.head()

,Unnamed: 0,BGGId,Rating,Username
0,20,213788,7.5,95783
1,26,213788,7.0,355841
2,48,213788,7.0,343988
3,53,213788,7.0,54058
4,62,213788,6.0,129193


# Suprise!

In [35]:
!pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 4.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357275 sha256=b27344efd30e5cce8e40f228b30da17c968df2b3af97da6a85e80a6e9102455a
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [36]:
from surprise import SVD, accuracy
from surprise import Dataset, Reader
from surprise.model_selection import cross_validate
from surprise.model_selection.split import train_test_split
from collections import defaultdict

In [1]:
# preprocessing the data
reader = Reader(rating_scale=(1,10))
data = Dataset.load_from_df(users_df[['Username','BGGId','Rating']], reader)
train, test = train_test_split(data, test_size=.2, random_state=42)

# initial model
algo = SVD(random_state = 42)
algo.fit(train)
predictions = algo.test(test)

# evaluate the rmse result of the prediction and ground thuth
accuracy.rmse(predictions)

NameError: name 'Reader' is not defined

In [40]:
def get_top_picks(predictions, num_recs = 10):
    """Return the top-N recommendation for each user from a set of predictions.

    predictions(list of Prediction objects): The list of predictions, as returned by suprise's algorithm.
    num_recs (int): The number of recommendation to output for each user. Default is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size num_recs.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:num_recs]

    return top_n

top_n = get_top_picks(predictions, num_recs=10)

# Print the recommended items for each user
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])

Streaming output truncated to the last 5000 lines.
155574 [185343]
383141 [432, 180020, 163412, 2471, 121921]
153903 [172225, 20806, 1234]
355591 [18602, 31260]
396011 [70323, 9209, 11]
289686 [42215, 18602, 327, 432, 73761, 66589, 12, 915, 11]
205584 [58798, 98778]
317283 [29368, 206206, 83330, 243759, 150997, 163399, 269385, 40603, 170771, 193037]
315156 [122298, 37589, 22497, 169318, 141572, 2223, 128666, 178900]
49013 [521, 237182, 54043, 199561, 148228, 188834, 178900]
380768 [25613, 21050, 14105, 19650, 17392]
102498 [27162]
17197 [147020, 68448, 187645]
309122 [2083, 2181, 3406, 527, 4112]
308755 [178900, 237182, 225910]
109730 [163967, 209685, 160851, 140933, 170]
237597 [1515, 236457]
316742 [29368, 34635, 163602]
230827 [54998]
81578 [160851, 50]
101855 [108745, 224316, 199493, 39339, 143986, 37380, 18, 245353, 478, 203416]
180308 [41114, 70323]
376325 [3076, 39856, 124361, 478, 38453, 126163]
276331 [9609]
218756 [28023, 54043, 206504, 178336, 174078]
151610 [205637, 30549, 